In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import copy
import gym
import numpy as np
import ray

gym.logger.setLevel(gym.logging.ERROR)

ray.init()

## Natural Evolution Strategies

An improved approach to derivative free optimization is to maintain a *population of policies*. This is sometimes done by maintaining a set of distinct policies. In this exercise, we will maintain a *distribution over policies*. The policies will use neural nets with a fixed architecture to choose actions. **The distribution over policies will be a multivariate Gaussian over the weights of the neural nets.** The Gaussian will be represented by its mean $\mu$. It will have a fixed covariance matrix $\sigma I$, where $I$ is the identity matrix.

The mean of the Gaussian will be initialized to the vector $\mu_0$ of all zeros (referred to as `mean_policy` in the code).

In [ ]:
# Initialize the mean policy to all zeros.
num_inputs = 4
num_hiddens = 5
num_outputs = 1

def initial_policy():
    mean_policy = {"weights1": np.zeros((num_hiddens, num_inputs)),
                   "biases1": np.zeros((num_hiddens)),
                   "weights2": np.zeros((num_outputs, num_hiddens)),
                   "biases2": np.zeros(num_outputs)}
    return mean_policy

We will use the helper function below to compute an action given a policy and an input state.

In [ ]:
def compute_action(policy, state):
    hiddens = np.maximum(np.dot(policy["weights1"], state) + policy["biases1"], 0)
    output = np.dot(policy["weights2"], hiddens) + policy["biases2"]
    assert output.size == 1
    # Turn output into a probability using a sigmoid function.
    probability_of_0 = 1 / (1 + np.exp(-output[0]))
    return 0 if np.random.uniform(0, 1) < probability_of_0 else 1

At time $t$, we will generate an updated mean vector $\mu_t$ as follows. We will generate $N$ samples $\epsilon_1, \ldots, \epsilon_N$ from a standard multivate normal.

\begin{equation}
\epsilon_n \sim \mathcal N(0, I) \quad \text{for $1 \le n \le N$}
\end{equation}

For each **noise vector** $\epsilon_n$, we will derive a **perturbed policy** $\mu_t + \sigma\epsilon_n$. Note that **this perturbed policy is a sample from the distribution over policies**.

For variance reduction, we will generate perturbed policies in pairs, with opposite perturbations. This is shown in the next box.

In [ ]:
sigma = 0.02

def generate_perturbed_policies(policy):
    new_policy1 = dict()
    new_policy2 = dict()
    for key, weights in policy.items():
        perturbation = sigma * np.random.normal(size=weights.shape)
        new_policy1[key] = weights + perturbation
        new_policy2[key] = weights - perturbation
    return new_policy1, new_policy2

For each of the $N$ perturbed policies, we will simulate some rollouts and compute the average reward of these rollouts $R_n$.

In [ ]:
def rollout_policy(env, policy):
    state = env.reset()
    done = False
    cumulative_reward = 0
    while not done:
        action = compute_action(policy, state)
        state, reward, done, _ = env.step(action)
        cumulative_reward += reward
    return cumulative_reward

def evaluate_perturbed_policies(mean_policy, N):
    policy1, policy2 = generate_perturbed_policies(mean_policy)
    env = gym.make("CartPole-v0")
    #env = TestEnvironment1()
    reward1 = rollout_policy(env, policy1)
    reward2 = rollout_policy(env, policy2)
    return [policy1, policy2], [reward1, reward2]

We will then **update the mean of the distribution over policies by moving it in the direction of the perturbed policies that performed better**. That is,

\begin{equation}
\mu_{t+1} = \mu_t + \frac{\alpha}{N \sigma} \sum_{n=1}^N R_n \epsilon_n,
\end{equation}

where $\alpha$ is the learning rate.

**This is a stochastic estimate of the gradient of the expected reward of a policy generated from the random distribution taken with respect to the mean of the distribution.**

In [ ]:
alpha = 0.01

def update_mean_policy(mean_policy, perturbed_policies, rewards):
    new_mean_policy = copy.deepcopy(mean_policy)
    N = len(perturbed_policies)
    for policy, reward in zip(perturbed_policies, rewards):
        for key in mean_policy:
            new_mean_policy[key] += (alpha / (N * sigma)) * (policy[key] - mean_policy[key]) * reward
    return new_mean_policy

Putting it all together, we learn a policy by iteratively improving the mean of a distribution of policies. We improve the mean by sampling policies from the distribution over policies and moving the mean in the direction of the policies that performed better.

In code, we have the following.

**Exercise:** Parallelize the algorithm below using Ray. You may want to turn `evaluate_perturbed_policies` into a remote function.

In [ ]:
mean_policy = initial_policy()

num_iters = 10
for _ in range(num_iters):
    perturbed_policies_and_rewards = [evaluate_perturbed_policies(mean_policy, 100)
                                      for _ in range(100)]
    all_perturbed_policies = []
    all_rewards = []
    for perturbed_policies, rewards in perturbed_policies_and_rewards:
        all_perturbed_policies.extend(perturbed_policies)
        all_rewards.extend(rewards)
    print("Average reward is {}.".format(np.mean(all_rewards)))
    
    mean_policy = update_mean_policy(mean_policy, all_perturbed_policies, all_rewards)

**Note about efficiency:** In the above example, we're returning the entire perturbation vectors. In the cluster setting, this may require shipping fairly large parameter vectors across the network, which can be expensive. It turns out that it's unnecessary for this algorithm. Because the perturbation vectors are generated randomly, it suffices to ship the seed that was used to generate the perturbations so that they can be regenerated on the other side. This strategy can be used to eliminate nearly all required communication for this algorithm.